## Instructions

We first downloaded the files from this link <http://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip> 

Then we dragged the pubmed folder into that same folder, with the copied abstracts (if you don't want to re-download them all again).

There are a few files you will need to make sure are present:

`lexparser-gui.bat                  
lexparser-gui.command              
lexparser-gui.sh                   
lexparser-lang-train-test.sh       
lexparser-lang.sh                  
lexparser.bat                      
lexparser.sh                       `

You will also need to add the `edu` folder that can be found here:
<https://www.dropbox.com/s/t9uk4z1xznpo0jz/jars.zip?dl=0>

Add the .zip extension to the `stanford-corenlp-3.8.0-models.jar` file, and unzip it. Copy that `edu` folder and paste it in to your home directory.

# Parsing the Pubmed Abstracts

In [87]:
import pubmed.utils as pb
import json
import re
from collections import defaultdict
from pprint import pprint
import string
# utf-8 support
import codecs
import nltk
# spit abstracts to sentences
from nltk.tokenize import sent_tokenize
import ast
import pickle

#pandas!
import pandas as pd
import numpy as np
import re

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lisabarcelo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
!curl "https://docs.google.com/spreadsheets/d/1f70DUcOGq1KXx_Mwi_4CpIudBASF6WFgFjtDxmwN1Oc/pub?output=tsv" -o sent_files/glp_sent.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 92095    0 92095    0     0   101k      0 --:--:-- --:--:-- --:--:--  104k


In [4]:
!head sent_files/glp_sent.tsv

In [49]:
%%writefile sent_files/for_SSA.py

import re 
for item in open("sent_files/glp_sent.tsv","r"):
    sent_id, sentiment, compound, sentence = item.split("\t",3)
    sentence = re.sub(r'\([^)]*\)', '', sentence)
    sentence = sentence.replace(";"," ")
    #sentence = re.sub("\d+","",re.sub(r'[^\w\s]','',sentence))
    words_ = [w for w in sentence.split(" ")]
    if len(words_) >= 50:
        print " ".join([w for w in words_][:50])
    else:
        print sentence.encode('utf-8')

Overwriting sent_files/for_SSA.py


In [50]:
!python2 sent_files/for_SSA.py > sent_files/ssa_glp.txt

In [51]:
!head sent_files/ssa_glp.txt

In [52]:
!chmod a+x lexparser.sh

# Testing with PCFG Model*

*notice I made a change to the lexparser file to allow for more memory

In [8]:
# !cat ./lexparser.sh

In [15]:
# %%timeit
# ! ./lexparser.sh  sent_files/ssa_acetaminophen.txt

## Testing with RNN Model

In [9]:
# !cat ./lexparser_rnn.sh

In [17]:
# %%timeit
# ! ./lexparser_rnn.sh  sent_files/ssa_acetaminophen.txt

## Test with Caseless PCFG  Model

In [10]:
# !cat ./lexparser_caseless.sh

In [11]:
# %%timeit
# ! ./lexparser_caseless.sh  sent_files/ssa_glp.txt

## Command Line Sentiment Analysis

This creates an output file with tuples and sentiments!

In [53]:
!java -cp "*" -Xmx2g edu.stanford.nlp.pipeline.StanfordCoreNLP \
-annotators tokenize,ssplit,pos,lemma,ner,parse,dcoref,sentiment \
-file sent_files/ssa_glp.txt \
-outputFormat text

[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [1.2 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.8 sec].
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.muc.7cla

In [64]:
!cat ssa_glp.txt.out > sent_files/output_file.json

In [65]:
!head sent_files/output_file.json

Sentence #1 (27 tokens, sentiment: Very negative):


Casein , safflower oil , sucrose  and rebaudioside A  stimulated GLP-1 release from intestinal segments, whereas casein only stimulated PYY and CCK release."
[Text=sentence CharacterOffsetBegin=0 CharacterOffsetEnd=8 PartOfSpeech=NN Lemma=sentence NamedEntityTag=O SentimentClass=Neutral]
[Text=Casein CharacterOffsetBegin=11 CharacterOffsetEnd=17 PartOfSpeech=NN Lemma=casein NamedEntityTag=O SentimentClass=Neutral]
[Text=, CharacterOffsetBegin=18 CharacterOffsetEnd=19 PartOfSpeech=, Lemma=, NamedEntityTag=O SentimentClass=Neutral]
[Text=safflower CharacterOffsetBegin=20 CharacterOffsetEnd=29 PartOfSpeech=NN Lemma=safflower NamedEntityTag=O SentimentClass=Neutral]
[Text=oil CharacterOffsetBegin=30 CharacterOffsetEnd=33 PartOfSpeech=NN Lemma=oil NamedEntityTag=O SentimentClass=Neutral]
[Text=, CharacterOffsetBegin=34 CharacterOffsetEnd=35 PartOfSpeech=, Lemma=, NamedEntityTag=O SentimentClass=Neutral]


## Creating Dictionaries

In [144]:
h = ['happy','sad','very sad']

for i in h:
    i = i.replace("very sad","sad!")
    print i

happy
sad
sad!


In [145]:
from nltk.corpus import stopwords

#Creating Dictionary
co_occurrence_dict = defaultdict(list)

#List of Stopwords
stop_words = [w for w in stopwords.words('english')]

#Picking out the relevant output from the sentiment analyzer
for line in open('sent_files/output_file.json').readlines():
    if 'Sentence #' in line:
        sentence = str(line.strip('\n')).split(" ")[1]
        sentiment = str(line.strip('\n')).split(":")[1]
        sentiment = re.sub("\d+","",re.sub(r'[^\w\s]','',sentiment))
    elif line[0:5] == '[Text':
        word = str(line.split("=")[1]).split(" ")[0].lower()
        pos = str(line.split("=")[4]).split(" ")[0]
        
        #Ignoring punctuation, small words, and stop words
        if (len(word)>2  and word not in stop_words):
            
            #No need for redundancy
            sentiment = sentiment.replace("Very negative","Negative")
            
            #Adding to dictionary
            co_occurrence_dict[(sentiment, sentence)].append(word)

In [146]:
#Example of one of the entries in the dictionary

from itertools import islice
import  pprint
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

n_items = take(1, co_occurrence_dict.iteritems())
pp = pprint.PrettyPrinter(depth=6)
pp.pprint(n_items)

[((' Negative', '#137'),
  ['plasma',
   'glucagon-like',
   'peptide-1',
   'concentration',
   'mesenteric',
   'vein',
   'draining',
   'looped',
   'region',
   'intestine',
   'increased',
   'infusion',
   '139',
   'mmol/l',
   'solutions',
   'd-glucose',
   'd-galactose',
   'd-glucuronic',
   'acid',
   '3-0-methyl-d-glucose',
   'maltose',
   'sucrose',
   'maltitol',
   'intestinal',
   'lumen',
   'infusion',
   'solutions',
   'd-fructose',
   'd-fucose',
   'd-mannose',
   'd-xylose',
   'determine',
   'whether',
   'glucagon-like',
   'peptide',
   'analogue',
   'exendin-4',
   'augments',
   'neurochemical',
   'effects',
   'single',
   'l-dopa',
   'treatment',
   'whether',
   'ex-4',
   'decrease',
   'l-dopa',
   'induced',
   'dyskinesias'])]


In [148]:
sent_dict = defaultdict(list)
words_dict = defaultdict(list)


# Assuming these are the key words we are curious about.
# search_phrase = ['australian','statin']

#Searching through the dictionary
for k,v in co_occurrence_dict.iteritems():
    for vv in v:
        if "glucagon-like" in v:
            words_dict[(vv, "GLP-1")].append(k[0])

for k, v in words_dict.iteritems():
    for vv in set(v):
        if "-" not in vv:
            sent_dict[k].append({vv: v.count(vv)})

In [129]:
## List of Food Compounds
# food_list = list(set([word.lower() for word in pickle.load(open( "data/food_common.pickle", "rb"))]))

In [149]:
for k, v in sent_dict.iteritems():
    for vv in v:
        print k, vv

('isolation', 'GLP-1') {' Negative': 3}
('gip', 'GLP-1') {' Negative': 1}
('glucose-dependent', 'GLP-1') {' Negative': 7}
('preinjected', 'GLP-1') {' Negative': 1}
('tolerance', 'GLP-1') {' Negative': 2}
('synthesized', 'GLP-1') {' Negative': 1}
('supporting', 'GLP-1') {' Negative': 1}
('rats', 'GLP-1') {' Negative': 5}
('rats', 'GLP-1') {' Neutral': 1}
('minocycline', 'GLP-1') {' Negative': 1}
('postmeal', 'GLP-1') {' Negative': 1}
('anti-ctla-4', 'GLP-1') {' Negative': 3}
('lengths', 'GLP-1') {' Negative': 1}
('dose', 'GLP-1') {' Negative': 3}
('including', 'GLP-1') {' Negative': 2}
('flavonols', 'GLP-1') {' Negative': 1}
('specifically', 'GLP-1') {' Negative': 3}
('fish', 'GLP-1') {' Negative': 3}
('receptor/p38', 'GLP-1') {' Negative': 1}
('type-2', 'GLP-1') {' Negative': 1}
('origin', 'GLP-1') {' Negative': 1}
('macromolecule', 'GLP-1') {' Negative': 1}
('chicken', 'GLP-1') {' Negative': 3}
('prostaglandin', 'GLP-1') {' Negative': 1}
('almond', 'GLP-1') {' Negative': 1}
('sequence

In [157]:
sent_dict.values()

[[{' Negative': 3}],
 [{' Negative': 1}],
 [{' Negative': 7}],
 [{' Negative': 1}],
 [{' Negative': 2}],
 [{' Negative': 1}],
 [{' Negative': 1}],
 [{' Negative': 5}, {' Neutral': 1}],
 [{' Negative': 1}],
 [{' Negative': 1}],
 [{' Negative': 3}],
 [{' Negative': 1}],
 [{' Negative': 3}],
 [{' Negative': 2}],
 [{' Negative': 1}],
 [{' Negative': 3}],
 [{' Negative': 3}],
 [{' Negative': 1}],
 [{' Negative': 1}],
 [{' Negative': 1}],
 [{' Negative': 1}],
 [{' Negative': 3}],
 [{' Negative': 1}],
 [{' Negative': 1}],
 [{' Negative': 3}],
 [{' Negative': 1}],
 [{' Negative': 2}],
 [{' Negative': 3}],
 [{' Negative': 3}],
 [{' Negative': 2}, {' Neutral': 1}],
 [{' Negative': 2}],
 [{' Negative': 2}],
 [{' Negative': 2}],
 [{' Negative': 2}],
 [{' Negative': 4}],
 [{' Negative': 2}],
 [{' Negative': 1}],
 [{' Negative': 1}],
 [{' Negative': 2}],
 [{' Negative': 1}],
 [{' Negative': 2}],
 [{' Negative': 1}],
 [{' Negative': 4}],
 [{' Negative': 1}],
 [{' Negative': 1}],
 [{' Negative': 1}],
